In [6]:
search_for = 120
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.006737709045410156
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 875
all cases: 1116151311
type: [1, 1, 1, 1, 120] 120
cases of this type: 207360000
10000 77.52491493973291
20000 79.92104107640918
30000 78.28158232104796
40000 78.58743007658467
50000 288.1390879467526
60000 386.06330857340606
70000 397.96672187689336
80000 337.6634554048372
90000 379.9764762352775
100000 411.96193235991626
110000 426.7400010776933
120000 464.6630663287706
130000 429.68364152650804
140000 484.2151775465835
150000 465.6604503130966
160000 387.6089184453878
170000 529.171409514258
180000 475.9156854291485
190000 509.14246175607985
200000 497.9209198097649
210000 520.8961070351686
220000 616.5655920138797
230000 684.0035986104715
240000 734.1520382610582
250000 886.1063466832887
260000 798.0438810343188
270000 826.460205316004
280000 821.9782194958988
290000 806.1419561744957
300000 794.6525994283887
310000 781.1373231867116
3

3130000 749.0168053090263
3140000 756.1670075793386
3150000 784.0286579230765
3160000 757.9341107451244
3170000 743.4478284128984
3180000 758.6225414090512
3190000 721.08376209596
3200000 773.5269804161825
3210000 787.3808074114123
3220000 756.3677179460191
3230000 793.464469458352
3240000 770.7232719937155
3250000 761.8735043044268
3260000 791.8723258016983
3270000 739.2311414317516
3280000 800.6317280770407
3290000 771.7752030658942
3300000 737.1493925467856
3310000 834.9911137045714
3320000 723.5903451775921
3330000 785.8230602283139
3340000 795.248753769695
3350000 736.9906472235164
3360000 793.665277068965
3370000 733.014698824004
3380000 770.2381909495109
3390000 808.3902302185743
3400000 741.967727222931
3410000 918.3111489356158
3420000 407.42326993833757
3430000 244.1781104821644
3440000 249.03833991003484
3450000 242.9115665953517
3460000 232.27634827512824
3470000 195.7688973802914
3480000 194.49873123697262
3490000 201.27696506996398
3500000 196.4281373244844
3510000 195.92

6290000 825.6025519505163
6300000 759.3232607701127
6310000 757.6647508805622
6320000 815.6596211059297
6330000 767.4234746657753
6340000 756.6421802560754
6350000 790.2484229152647
6360000 774.7906524926016
6370000 755.1380540397279
6380000 769.2049954671219
6390000 828.894307754194
6400000 796.6860174761866
6410000 779.1877890864786
6420000 782.80799086314
6430000 775.7649872911982
6440000 812.2158575854836
6450000 842.8379154527971
6460000 755.7910681435211
6470000 738.1476192919279
6480000 824.5991845787075
6490000 957.1284063008551
6500000 847.5310853185032
6510000 830.1942308894292
6520000 832.8610282024046
6530000 869.257191211787
6540000 830.8258686102139
6550000 779.3351906514827
6560000 814.3920518155937
6570000 836.7249033000916
6580000 860.4524606246472
6590000 813.0261844507695
6600000 806.2427545595655
6610000 791.8091090534306
6620000 817.4557393196421
6630000 799.1683593270112
6640000 792.214905510365
6650000 838.1764948785002
6660000 796.2023929799559
6670000 784.55475

9450000 813.0259468654925
9460000 867.4324545092883
9470000 873.3319346105174
9480000 563.0724537288661
9490000 96.48532038212099
9500000 96.3535881200268
9510000 96.644360173021
9520000 96.45405601783408
9530000 96.46990155449252
9540000 97.98529196812187
9550000 302.0855531094677
9560000 1125.2499300117543
9570000 782.5223184824794
9580000 850.6703473566477
9590000 758.1226755433241
9600000 735.8871683440054
9610000 824.7524792615848
9620000 737.6166836501161
9630000 872.0944743096554
9640000 827.8491343911132
9650000 777.7706673315967
9660000 890.9561841285771
9670000 786.2088998581901
9680000 825.636763084081
9690000 887.6638494025079
9700000 783.0639447724236
9710000 831.1832097206294
9720000 791.383643994276
9730000 756.2383122761779
9740000 792.5596647751972
9750000 762.6053770444131
9760000 796.7635998806467
9770000 784.1522050505029
9780000 759.8900524853866
9790000 801.6239745787044
9800000 743.9931731405526
9810000 775.5161506800268
9820000 804.9169100783186
9830000 787.1191

12510000 901.4589206993138
12520000 842.0908540052321
12530000 830.0095093781679
12540000 792.4215518927856
12550000 820.0642742261123
12560000 857.4042305173577
12570000 879.9364099022685
12580000 769.4164303645559
12590000 752.8832497194178
12600000 790.151911584705
12610000 900.125936001626
12620000 795.8753716919404
12630000 780.9839723688003
12640000 800.6842719652926
12650000 836.4331028671138
12660000 786.1306147751882
12670000 811.2503457556247
12680000 756.2740466713215
12690000 792.5832769672619
12700000 790.9444005236986
12710000 781.2176134072156
12720000 808.8988760544496
12730000 793.7437233974352
12740000 800.2385658966602
12750000 828.4287501153243
12760000 793.8157078328614
12770000 855.9860884578213
12780000 822.2527303140886
12790000 797.8414773464863
12800000 850.8725926093342
12810000 804.132507855367
12820000 852.0112704623705
12830000 816.9025947931342
12840000 764.0386642134821
12850000 827.2668385423153
12860000 760.16943464644
12870000 803.6413870127212
128800

...

124290000 648.4459470220831
124300000 616.1649583904554
124310000 639.2774691545063
124320000 568.8656095243939
124330000 589.6755241807915
124340000 656.7431499483779
124350000 695.2120948878015
124360000 635.5384591316468
124370000 647.5468791089867
124380000 650.8068220407007
124390000 591.645391779162
124400000 746.8720941628345
124410000 701.6284007648223
124420000 700.4224469492954
124430000 663.5224840463269
124440000 631.5192976753034
124450000 705.3886262916496
124460000 629.4535671875714
124470000 657.7526437120443
124480000 618.7521368736365
124490000 562.8665318271571
124500000 620.8771714809044
124510000 619.2543167926096
124520000 653.8962194574129
124530000 722.6577858871503
124540000 549.4125525862436
124550000 631.9154692141452
124560000 583.6754097470763
124570000 583.9105449270087
124580000 633.5371874096346
124590000 610.963643712267
124600000 600.8098163142388
124610000 608.6641060833346
124620000 573.282340689384
124630000 614.5741739201695
124640000 579.611625465

127220000 164.34238805753876
127230000 130.94567393342268
127240000 243.8552980377277
127250000 574.1016017999826
127260000 617.4959169817092
127270000 616.4957088752591
127280000 464.9628261587477
127290000 597.5131803840698
127300000 592.2431948364167
127310000 427.2862787911303
127320000 455.83480229957735
127330000 631.1586196060285
127340000 581.0729968062511
127350000 578.4769045786092
127360000 719.9414830541236
127370000 639.8148653596278
127380000 396.63947698552107
127390000 566.2445640721385
127400000 471.3151139621408
127410000 416.29382535195447
127420000 672.7400858161604
127430000 606.8167636329931
127440000 594.5035149429409
127450000 468.21775006152166
127460000 585.6127176617086
127470000 595.9366928906309
127480000 556.4915800691881
127490000 627.4892569375108
127500000 626.7065243629477
127510000 565.6101307393793
127520000 649.602124036611
127530000 634.3271673793138
127540000 483.0080425818147
127550000 485.5480393329585
127560000 666.2141215952835
127570000 582.2

130140000 496.15832653810446
130150000 488.21363515403897
130160000 457.2553492770395
130170000 538.104415424302
130180000 475.0582227953841
130190000 436.4758477249957
130200000 459.62818991176607
130210000 627.0427912253239
130220000 514.8534889017517
130230000 157.21951910647408
130240000 165.91168768366236
130250000 171.1981191015578
130260000 177.46094578775595
130270000 118.91668285179053
130280000 153.71035152044334
130290000 153.5137746565964
130300000 568.9209096582758
130310000 560.5377990673144
130320000 433.7989478446657
130330000 469.899039809038
130340000 465.9813722529359
130350000 355.60922888057604
130360000 398.2673713960162
130370000 571.0517011325608
130380000 514.7996546327557
130390000 410.2983804613615
130400000 493.83091314644656
130410000 562.1125991149643
130420000 445.79491970777326
130430000 496.7179904822702
130440000 425.3928550701959
130450000 54.73150093822989
130460000 51.77398815767125
130470000 66.19826259001537
130480000 49.18231739716203
130490000 6

133060000 573.7229263921994
133070000 665.3195400599623
133080000 547.4474102578884
133090000 499.6142877698494
133100000 546.087650478459
133110000 508.5052940364089
133120000 450.418219280497
133130000 568.4511002760922
133140000 363.45642384271287
133150000 588.1154011105444
133160000 548.2479160919578
133170000 686.4145383829119
133180000 477.67044901707953
133190000 686.9355881737217
133200000 414.95502719426935
133210000 588.4036771639809
133220000 615.7857693071868
133230000 567.1391864918436
133240000 622.1913252129959
133250000 613.8136112388108
133260000 615.9993914903271
133270000 574.0729002251517
133280000 528.1327947149165
133290000 620.4633165219426
133300000 586.5898727360376
133310000 657.983910528437
133320000 600.1428489131143
133330000 594.7512938366942
133340000 547.5760878842091
133350000 465.86023079601665
133360000 568.0628569218587
133370000 613.0072046285729
133380000 644.3142420651084
133390000 625.8916778666288
133400000 602.3910323588369
133410000 610.91720

KeyboardInterrupt: 